In [14]:
# 44. Wildcard Matching
# recursion, still get TLE...
class Solution:
    def __init__(self):
        self.memo = {}
        
    def isMatch(self, s, p):
        p = self.mergeStars(p)
        return self.helper(s, p)
    
    def mergeStars(self, p):
        if p == '':
            return p
        new = [p[0]]
        for i in range(1, len(p)):
            if new[-1] != '*' or (new[-1] == '*' and p[i] != '*'):
                new += [p[i]]
        return ''.join(new)
    
    def helper(self, s, p):
        if s == p or p == '*':
            self.memo[(s, p)] = True
        elif p == '' or s == '':
            self.memo[(s, p)] = False
        elif p[0] == s[0] or p[0] == '?':
            self.memo[(s, p)] = self.helper(s[1:], p[1:])
        elif p[0] == '*':
            self.memo[(s, p)] = self.helper(s, p[1:]) or self.helper(s[1:], p)
        else:
            self.memo[(s, p)] = False
        
        return self.memo[(s, p)]

s = "bbaaaabaaaaabbabbabbabbababaabababaabbabaaabbaababababbabaabbabbbbbbaaaaaabaabbbbbabbbbabbabababaaaaa"
p = "******aa*bbb*aa*a*bb*ab***bbba*a*babaab*b*aa*a****"

Solution().isMatch(s, p)

False

In [ ]:
# solution 2, dp
class Solution:
    def isMatch(self, s, p):
        s_len = len(s)
        p_len = len(p)
        
        # base cases
        if p == s or p == '*':
            return True
        if p == '' or s == '':
            return False
        
        # init all matrix except [0][0] element as False
        d = [ [False] * (s_len + 1) for _ in range(p_len + 1)]
        d[0][0] = True
        
        # DP compute 
        for p_idx in range(1, p_len + 1):
            # the current character in the pattern is '*'
            if p[p_idx - 1] == '*':
                s_idx = 1
                # d[p_idx - 1][s_idx - 1] is a string-pattern match 
                # on the previous step, i.e. one character before.
                # Find the first idx in string with the previous math.
                while not d[p_idx - 1][s_idx - 1] and s_idx < s_len + 1:
                    s_idx += 1
                # If (string) matches (pattern), 
                # when (string) matches (pattern)* as well
                d[p_idx][s_idx - 1] = d[p_idx - 1][s_idx - 1]
                # If (string) matches (pattern), 
                # when (string)(whatever_characters) matches (pattern)* as well
                while s_idx < s_len + 1:
                    d[p_idx][s_idx] = True
                    s_idx += 1
            # the current character in the pattern is '?'
            elif p[p_idx - 1] == '?':
                for s_idx in range(1, s_len + 1): 
                    d[p_idx][s_idx] = d[p_idx - 1][s_idx - 1] 
            # the current character in the pattern is not '*' or '?'
            else:
                for s_idx in range(1, s_len + 1): 
                    # Match is possible if there is a previous match
                    # and current characters are the same
                    d[p_idx][s_idx] = \
                    d[p_idx - 1][s_idx - 1] and p[p_idx - 1] == s[s_idx - 1]  
                                                               
        return d[p_len][s_len]

In [ ]:
# solution 1, recursion w/ memo
class Solution:
    def remove_duplicate_stars(self, p):
        if p == '':
            return p
        p1 = [p[0],]
        for x in p[1:]:
            if p1[-1] != '*' or p1[-1] == '*' and x != '*':
                p1.append(x)
        return ''.join(p1) 
        
    def helper(self, s, p):
        dp = self.dp
        if (s, p) in dp:
            return dp[(s, p)]

        if p == s or p == '*':
            dp[(s, p)] = True
        elif p == '' or s == '':
            dp[(s, p)] = False
        elif p[0] == s[0] or p[0] == '?':
            dp[(s, p)] = self.helper(s[1:], p[1:])
        elif p[0] == '*':
            dp[(s, p)] = self.helper(s, p[1:]) or self.helper(s[1:], p)
        else:
            dp[(s, p)] = False

        return dp[(s, p)]
        
    def isMatch(self, s, p):
        p = self.remove_duplicate_stars(p)
        # memorization hashmap to be used during the recursion
        self.dp = {}
        return self.helper(s, p)

In [11]:
# 44. Wildcard Matching, failed
class Solution:
    def isMatch(self, s, p):
        i_s = 0
        i_p = 0
        while i_s < len(s) and i_p < len(p):
            if s[i_s] == p[i_p] or p[i_p] == '?':
                i_s += 1
                i_p += 1
            elif p[i_p] == '*':
                i_s += 1
            else:
                return False
        print(s[i_s:])
        print(p[i_p:])
        if i_s < len(s) and i_p == len(p):
            return False
        elif i_s == len(s) and i_p == len(p)-1 and p[i_p] == '*':
            return True
        else:
            return False
        return True

s = "adceb"
p = "*a*b"

Solution().isMatch(s, p)


*a*b


False

In [ ]:
# solution 3, dp improved
class Solution:
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        s_len, p_len = len(s), len(p)
        s_idx = p_idx = 0
        star_idx = s_tmp_idx = -1
 
        while s_idx < s_len:
            # If the pattern caracter = string character
            # or pattern character = '?'
            if p_idx < p_len and p[p_idx] in ['?', s[s_idx]]:
                s_idx += 1
                p_idx += 1
            # If pattern character = '*'
            elif p_idx < p_len and p[p_idx] == '*':
                # Check the situation
                # when '*' matches no characters
                star_idx = p_idx
                s_tmp_idx = s_idx
                p_idx += 1
            # If pattern character != string character
            # or pattern is used up
            # and there was no '*' character in pattern 
            elif star_idx == -1:
                return False
            # If pattern character != string character
            # or pattern is used up
            # and there was '*' character in pattern before
            else:
                # Backtrack: check the situation
                # when '*' matches one more character
                p_idx = star_idx + 1
                s_idx = s_tmp_idx + 1
                s_tmp_idx = s_idx
        
        # The remaining characters in the pattern should all be '*' characters
        return all(x == '*' for x in p[p_idx:])